In [1]:
from typing import Optional
import json
import random
import time
import datetime as dt
from datetime import datetime, timedelta
from dateutil.parser import parse

In [2]:
# Banks database tables
users = []
accounts = []
transactions = []
ledger_entries = []
bills = []

In [3]:
# User
class User:
    def __init__(self, name, email, password):
        self.user_id = str(len(users) + 1)
        self.name = name
        self.email = email
        self.password = password

    def __repr__(self):
        return f"User {self.user_id}: {self.name} ({self.email})"

class UserManager:
    def __init__(self):
        pass

    def add_user(self, name, email, password):
        new_user = User(name, email, password)
        users.append(new_user)
        return new_user

    def get_user(self, user_id):
        for user in users:
            if user.user_id == str(user_id):
                return user
        return None

    def get_all_users(self):
        return users

In [4]:
# User bank Accounts Management
class Account:
    def __init__(self, user_id, account_number, account_balance, account_type):
        self.account_id = str(len(accounts) + 1)
        self.user_id = user_id
        self.account_number = account_number
        self.account_balance = account_balance
        self.account_type = account_type

    @property
    def balance(self):
        return self.account_balance

    def deposit(self, amount):
        self.account_balance += amount

    def withdraw(self, amount):
        if amount > self.account_balance:
            print(f"Insufficient funds. Current balance is {self.account_balance}")
            return False
        else:
            self.account_balance -= amount
            return True

    def __repr__(self):
        return f"Account {self.account_id}: {self.account_number} ({self.account_type}) - Balance: {self.account_balance}"

class AccountManager:
    def __init__(self):
        pass

    def add_account(self, user_id, account_number, account_balance, account_type):
        new_account = Account(user_id, account_number, account_balance, account_type)
        accounts.append(new_account)
        return new_account

    def get_account(self, account_id):
        for account in accounts:
            if account.account_id == str(account_id):
                return account
        return None
        
    def get_account_balance(self, account_id):
        account = self.get_account(account_id)
        return account.balance

    def trace_account_balance(self, account_id, lt_date):
        balance = 0.0
        for transaction in transactions:
            if transaction.account_id == str(account_id) and transaction.transaction_date <= lt_date:
                if transaction.transaction_type in ["deposit", "transfer in"]:
                    balance += transaction.amount
                elif transaction.transaction_type in ["withdrawal", "transfer out", "bill"]:
                    balance -= transaction.amount
        return balance
        
    def get_all_accounts(self):
        return accounts

In [5]:
# User accounts Transactions Management
class Transaction:
    def __init__(self, account_id, transaction_type, amount):
        self.transaction_id = str(len(transactions) + 1)
        self.account_id = account_id
        self.transaction_type = transaction_type
        self.amount = amount
        self.transaction_date = datetime.now()

    def __repr__(self):
        return f"Transaction {self.transaction_id}: Account {self.account_id} - {self.transaction_type} - Amount: {self.amount} - Date: {self.transaction_date}"

class TransactionManager:
    def __init__(self):
        pass

    def add_transaction(self, account_id, transaction_type, amount):
        new_transaction = Transaction(account_id, transaction_type, amount)
        transactions.append(new_transaction)
        return new_transaction

    def get_transaction(self, transaction_id):
        for transaction in transactions:
            if transaction.transaction_id == str(transaction_id):
                return transaction
        return None

    def get_all_transactions(self):
        return transactions

    def get_transactions_by_account(self, account_id, gt_date, lt_date):
        _t = []
        for transaction in transactions:
            if transaction.account_id == str(account_id) and gt_date <= transaction.transaction_date <= lt_date:
                _t.append(transaction)
        return _t

In [6]:
# User ledger account activity tracker Management

class LedgerEntry:
    def __init__(self, account_id, transaction_id, debit_credit, amount):
        self.ledger_id = str(len(ledger_entries) + 1)
        self.account_id = account_id
        self.transaction_id = transaction_id
        self.debit_credit = debit_credit
        self.amount = amount

    def __repr__(self):
        return f"Ledger Entry {self.ledger_id}: Account {self.account_id} - Transaction {self.transaction_id} - {self.debit_credit} - Amount: {self.amount}"

class LedgerManager:
    def __init__(self):
        pass

    def add_ledger_entry(self, account_id, transaction_id, debit_credit, amount):
        new_ledger_entry = LedgerEntry(account_id, transaction_id, debit_credit, amount)
        ledger_entries.append(new_ledger_entry)
        return new_ledger_entry

    def get_ledger_entry(self, ledger_id):
        for ledger_entry in ledger_entries:
            if ledger_entry.ledger_id == str(ledger_id):
                return ledger_entry
        return None

    def get_all_ledger_entries(self):
        return ledger_entries

    def get_ledger_entries_by_account(self, account_id):
        return [ledger_entry for ledger_entry in ledger_entries if ledger_entry.account_id == str(account_id)]

In [7]:
# User utility bills payments Management

class Bill:
    def init(self, account_id, bill_type, amount):
        self.bill_id = str(len(bills) + 2)
        self.account_id = account_id
        self.bill_type = bill_type
        self.amount = amount
        
    def __repr__(self):
        return f"Bill {self.bill_id}: Account {self.account_id} - Type {self.bill_type} - Amount: {self.bill_amount}"

class BillManager:
    def __init__(self):
        pass

    def add_bill(self, account_id, bill_type, amount):
        bill = Bill(account_id, bill_type, amount)
        bills.appped(bill)
        return bill

    def get_bill(self, bill_id):
        for bill in bills:
            if bill.bill_id == str(bill_id):
                return bill
        return None

    def get_all_bills(self):
        return bills

In [8]:
# The Banking System with bank statements

class BankStatement:
    def __init__(self, account_id, start_date, end_date, start_balance, end_balance, transactions):
        self.account_id = account_id
        self.start_date = start_date
        self.end_date = end_date
        self.start_balance = start_balance
        self.end_balance = end_balance
        self.transactions = transactions

    def __str__(self):
        statement = f"Bank Statement for Account {self.account_id}\n"
        statement += f"Period: {self.start_date} - {self.end_date}\n"
        statement += f"Starting Balance: {self.start_balance}\n"
        statement += f"Ending Balance: {self.end_balance}\n"
        statement += "Transactions:\n"
        for transaction in self.transactions:
            statement += f"  {transaction}\n"
        return statement
        
class BankingSystem:
    def __init__(self):
        self.user_manager = UserManager()
        self.account_manager = AccountManager()
        self.transaction_manager = TransactionManager()
        self.ledger_manager = LedgerManager()
        self.bill_manager = BillManager()

    def create_user(self, name, email, password):
        user = self.user_manager.add_user(name, email, password)
        return user

    def create_account(self, user_id, account_number, account_balance, account_type):
        account = self.account_manager.add_account(user_id, account_number, account_balance, account_type)
        return account

    def deposit(self, account_id, amount):
        account = self.account_manager.get_account(account_id)
        account.deposit(amount)
        transaction = self.transaction_manager.add_transaction(account_id, "deposit", amount)
        self.ledger_manager.add_ledger_entry(account_id, transaction.transaction_id, "debit", amount)
        return transaction

    def withdraw(self, account_id, amount):
        account = self.account_manager.get_account(account_id)
        success = account.withdraw(amount)
        transaction = None
        if success:
            transaction = self.transaction_manager.add_transaction(account_id, "withdrawal", amount)
            self.ledger_manager.add_ledger_entry(account_id, transaction.transaction_id, "credit", amount)
        return success, transaction

    def transfer(self, from_account_id, to_account_id, amount):
        from_account = self.account_manager.get_account(from_account_id)
        to_account = self.account_manager.get_account(to_account_id)
        success = from_account.withdraw(amount)
        if success:
            transaction = self.transaction_manager.add_transaction(from_account_id, "transfer out", amount)
            self.ledger_manager.add_ledger_entry(from_account_id, transaction.transaction_id, "credit", amount)
            to_account.deposit(amount)
            transaction = self.transaction_manager.add_transaction(to_account_id, "transfer in", amount)
            self.ledger_manager.add_ledger_entry(to_account_id, transaction.transaction_id, "debit", amount)
        return success

    def pay_bill(self, account_id, bill_type, amount):
        account = self.account_manager.get_account(account_id)
        success = account.withdraw(amount)
        if success:
            self.bill_manager.add_bill(account_id, bill_type, amount)
            transaction = self.transaction_manager.add_transaction(account_id, "bill", amount)
            self.ledger_manager.add_ledger_entry(account_id, transaction.transaction_id, "credit", amount)
        return success
        
    def get_account_balance(self, account_id):
        return self.account_manager.get_account_balance(account_id)

    def get_transaction_history(self, account_id, start_date, end_date):
        transactions = self.transaction_manager.get_transactions_by_account(account_id, start_date, end_date)
        return transactions

    def get_ledger_entries(self, account_id):
        ledger_entries = self.ledger_manager.get_ledger_entries_by_account(account_id)
        return ledger_entries

    def get_bank_statement(self, account_id, start_date, end_date):
        transactions = self.transaction_manager.get_transactions_by_account(account_id, start_date, end_date)
        start_balance = self.account_manager.trace_account_balance(account_id, start_date)
        end_balance = self.account_manager.trace_account_balance(account_id, end_date)
        bank_statement = BankStatement(
            account_id=account_id,
            start_date=start_date,
            end_date=end_date,
            start_balance=start_balance,
            end_balance=end_balance,
            transactions=transactions
        )
        return bank_statement

In [9]:
# Banking Simulation runs to generate sample data

banking_system = BankingSystem()
def run_simulation():    
    # Create a banking system
    st = datetime.now()
    
    # Create a user and account
    users = [
        banking_system.create_user("John Doe", "johndoe@example.com", "password123"),
        banking_system.create_user("Mary Jane", "maryj@example.com", "password123"),
        banking_system.create_user("Amini Doe", "amini@example.com", "password123"),
        banking_system.create_user("Oeter Jane", "oeter@example.com", "password123")
    ]
    accounts = [
        banking_system.create_account(u.user_id, f"123456789{i}", 0.0, "savings") for i, u in enumerate(users)
    ]
    
    # Set simulation parameters
    simulation_days = 30
    initial_balance = 1000.0
    transaction_probability = 0.5
    transfer_probability = 0.2
    
    # Start simulation
    current_day = 0
    fds = st
    while current_day < simulation_days:
        # Randomly decide whether to deposit or withdraw
        account = random.choice(accounts)
        if random.random() < transaction_probability:
            # Deposit
            amount = random.uniform(100.0, 500.0)
            banking_system.deposit(account.account_id, amount)
            # print(f"Day {current_day+1}: Deposited ${amount:.2f}")
        else:
            # Withdraw
            amount = random.uniform(50.0, 200.0)
            banking_system.withdraw(account.account_id, amount)
            # print(f"Day {current_day+1}: Withdrew ${amount:.2f}")
    
        if random.random() < transaction_probability:
            another = list(filter(lambda ac: ac.account_id != account.account_id, accounts))[0]
            banking_system.transfer(account.account_id, another.account_id, amount)
    
        # Print bank statement every 5 days
        if (current_day + 1) % 5 == 0:
            # for acc in accounts:
            #     print("Bank Statement:")
            #     print(banking_system.get_bank_statement(acc.account_id, fds, datetime.now()))
            fds = datetime.now()
    
        # Increment day and wait for 1 second
        current_day += 1
        time.sleep(1)
    
    print("Simulation complete!")

In [10]:
run_simulation()

Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 122.86191432058197
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Insufficient funds. Current balance is 0.0
Simulation complete!


In [11]:
users

[User 1: John Doe (johndoe@example.com),
 User 2: Mary Jane (maryj@example.com),
 User 3: Amini Doe (amini@example.com),
 User 4: Oeter Jane (oeter@example.com)]

In [12]:
accounts

[Account 1: 1234567890 (savings) - Balance: 1459.413211486812,
 Account 2: 1234567891 (savings) - Balance: 629.5189306765942,
 Account 3: 1234567892 (savings) - Balance: 0.0,
 Account 4: 1234567893 (savings) - Balance: 503.07099642719015]

In [13]:
bills

[]

In [14]:
transactions[:5]

[Transaction 1: Account 2 - deposit - Amount: 398.2385802411794 - Date: 2024-07-08 20:11:23.225865,
 Transaction 2: Account 2 - withdrawal - Amount: 147.00793147884642 - Date: 2024-07-08 20:11:24.226165,
 Transaction 3: Account 2 - withdrawal - Amount: 186.09453018131686 - Date: 2024-07-08 20:11:25.226458,
 Transaction 4: Account 2 - deposit - Amount: 481.31525634724414 - Date: 2024-07-08 20:11:28.228332,
 Transaction 5: Account 2 - deposit - Amount: 395.0269433761185 - Date: 2024-07-08 20:11:30.229454]

In [15]:
ledger_entries[:5]

[Ledger Entry 1: Account 2 - Transaction 1 - debit - Amount: 398.2385802411794,
 Ledger Entry 2: Account 2 - Transaction 2 - credit - Amount: 147.00793147884642,
 Ledger Entry 3: Account 2 - Transaction 3 - credit - Amount: 186.09453018131686,
 Ledger Entry 4: Account 2 - Transaction 4 - debit - Amount: 481.31525634724414,
 Ledger Entry 5: Account 2 - Transaction 5 - debit - Amount: 395.0269433761185]

In [16]:
def class_to_dict(obj, depth=0):
    """
    Converts a class instance to a dictionary, formatting datetime objects as strings in "dd-mm-Y H:M:S" format.
    Recursively handles nested objects up to a specified depth.
    
    :param obj: The class instance to convert.
    :param depth: The maximum depth to recurse into nested objects.
    :return: A dictionary representation of the class instance.
    """
    result = {}
    if not obj or depth < 0:
        return None
    if hasattr(obj, '__dict__'):
        for attr_name, attr_value in vars(obj).items():
            if isinstance(attr_value, datetime):
                # Convert datetime object to string in "dd-mm-Y H:M:S" format
                attr_value = attr_value.strftime("%d-%m-%Y %H:%M:%S")
            elif isinstance(attr_value, list) and depth > 0:
                # Recursively convert list items
                attr_value = [class_to_dict(item, depth - 1) for item in attr_value]
            elif isinstance(attr_value, dict) and depth > 0:
                # Recursively convert dictionary items
                attr_value = class_to_dict(attr_value, depth - 1)
            result[attr_name] = attr_value
    return result

In [17]:
def get_account_balance(account_id: str) -> str:
    """Use this function to get the current account balance from the bank account manager.

    Args:
        account_id (str): The account id associated with the account. Required

    Returns:
        str: JSON: The current account balance in the account
    """

    # Fetch top story IDs
    balance = banking_system.get_account_balance(account_id)

    return json.dumps({
        "account_id": account_id,
        "balance": balance
    })

In [18]:
def get_bank_statement(account_id: str, start_date: str, end_date: str) -> str:
    """Use this function to get the bank statement to reflect user activity on their account.

    Args:
        account_id (str): The account id associated with the account. Required
        start_date (str): The start date for the historical transactions. Required
        end_date (str): The End date for the historical transactions. Required

    Returns:
        str: JSON: The bank statement
    """

    # Fetch top story IDs
    b_statement = banking_system.get_bank_statement(account_id, parse(start_date), parse(end_date))

    return json.dumps({
        "account_id": account_id,
        "statement": class_to_dict(b_statement)
    })

In [19]:
def withdraw_money(account_id: str, amount: int) -> str:
    """Use this function to withdraw money from an account.

    Args:
        account_id (str): The account id associated with the account. Required
        amount (int): The amount to be withdrawn from the account. Required

    Returns:
        str: JSON The withdrawal status and corresponding transaction
    """

    # Fetch top story IDs
    success, transaction = banking_system.withdraw(account_id, amount)

    return json.dumps({
        "account_id": account_id,
        "success": success,
        "transaction": class_to_dict(transaction)
    })

In [20]:
def get_dates_last_n_days(n: int):
    """
    Use this function to get start date and end date given the last N days.
    
    Args:
        n: Number of days ago from today. Required
        
    Return: 
        str: JSON with start_date and end_date range.
    """
    start_date = datetime.now() - timedelta(days=int(n))
    end_date = datetime.now()
    return json.dumps({
        "start_date": start_date.strftime("%d-%m-%Y %H:%M:%S"),
        "end_date": end_date.strftime("%d-%m-%Y %H:%M:%S")
    })

In [21]:
def get_dates_last_n_hours(n: int):
    """
    Use this function to get start date and end date given the last N hours.
    
    Args:
        n: Number of hours that have passed from now. Required
        
    Return: 
        str: JSON with start_date and end_date range.
    """
    start_date = datetime.now() - timedelta(hours=int(n))
    end_date = datetime.now()
    return json.dumps({
        "start_date": start_date.strftime("%d-%m-%Y %H:%M:%S"),
        "end_date": end_date.strftime("%d-%m-%Y %H:%M:%S")
    })

In [22]:
def get_dates_from_to(start_date, end_date):
    """
    Use this function to get start date and end date as datetime objects given string start date and end date.
    
    Args:
        start_date: Start date as a string in "DD-MM-YYYY" format. Required
        end_date: Start date as a string in "DD-MM-YYYY" format. Required
        
    Return: 
        str: JSON with start_date and end_date range.
    """
    s_date = datetime.strptime(start_date, "%d-%m-%Y")
    e_date = datetime.strptime(end_date, "%d-%m-%Y")
    return  json.dumps({
        "start_date": s_date.strftime("%d-%m-%Y %H:%M:%S"),
        "end_date": e_date.strftime("%d-%m-%Y %H:%M:%S")
    })

In [23]:
def get_bank_statement(account_id: str, n_days_ago: Optional[int] = None, n_hours_ago: Optional[int] = None, start_date: Optional[str] = None, end_date: Optional[str] = None) -> str:
    """Use this function to get the bank statement to reflect user activity on their account.

    Args:
        account_id (str): The account id associated with the account. Required
        n_days_ago (Optional[int]): The number of days ago. Optional
        n_hours_ago (Optional[int]): The number of hours ago. Optional
        start_date (Optional[str]): The start date for the historical transactions. Optional
        end_date (Optional[str]): The End date for the historical transactions. Optional

    Returns:
        str: JSON: The bank statement
    """
    if n_days_ago:
        dates = get_dates_last_n_days(n_days_ago)
    elif n_hours_ago:
        dates = get_dates_last_n_hours(n_hours_ago)
    elif start_date and end_date:
        dates = get_dates_from_to(start_date, end_date)
    else:
        dates = None
        
    if dates:
        dates = json.loads(dates)
    else:
      dates = {
            "start_date": (datetime.now() - timedelta(days=30)).strftime("%d-%m-%Y %H:%M:%S"),
            "end_date": datetime.now().strftime("%d-%m-%Y %H:%M:%S")
        }

    # Fetch top story IDs
    print(dates)
    b_statement = banking_system.get_bank_statement(
        account_id, 
        parse(dates.get("start_date"), dayfirst=True), 
        parse(dates.get("end_date"), dayfirst=True)
    )
    
    print(b_statement)
    return json.dumps({
        "account_id": account_id,
        "statement": class_to_dict(b_statement, 5)
    })

In [24]:
get_bank_statement(account_id=1, n_hours_ago=5)

{'start_date': '08-07-2024 15:11:53', 'end_date': '08-07-2024 20:11:53'}
Bank Statement for Account 1
Period: 2024-07-08 15:11:53 - 2024-07-08 20:11:53
Starting Balance: 0.0
Ending Balance: 1459.413211486812
Transactions:
  Transaction 8: Account 1 - transfer in - Amount: 122.86191432058197 - Date: 2024-07-08 20:11:31.230127
  Transaction 12: Account 1 - transfer in - Amount: 216.39594483315858 - Date: 2024-07-08 20:11:35.232900
  Transaction 13: Account 1 - withdrawal - Amount: 185.72611847062743 - Date: 2024-07-08 20:11:38.233822
  Transaction 17: Account 1 - transfer in - Amount: 132.95964763990162 - Date: 2024-07-08 20:11:40.234437
  Transaction 20: Account 1 - transfer in - Amount: 100.15825967221213 - Date: 2024-07-08 20:11:43.235745
  Transaction 23: Account 1 - transfer in - Amount: 385.66284267204173 - Date: 2024-07-08 20:11:45.236572
  Transaction 26: Account 1 - transfer in - Amount: 448.4728356782504 - Date: 2024-07-08 20:11:46.237059
  Transaction 27: Account 1 - withdrawa

'{"account_id": 1, "statement": {"account_id": 1, "start_date": "08-07-2024 15:11:53", "end_date": "08-07-2024 20:11:53", "start_balance": 0.0, "end_balance": 1459.413211486812, "transactions": [{"transaction_id": "8", "account_id": "1", "transaction_type": "transfer in", "amount": 122.86191432058197, "transaction_date": "08-07-2024 20:11:31"}, {"transaction_id": "12", "account_id": "1", "transaction_type": "transfer in", "amount": 216.39594483315858, "transaction_date": "08-07-2024 20:11:35"}, {"transaction_id": "13", "account_id": "1", "transaction_type": "withdrawal", "amount": 185.72611847062743, "transaction_date": "08-07-2024 20:11:38"}, {"transaction_id": "17", "account_id": "1", "transaction_type": "transfer in", "amount": 132.95964763990162, "transaction_date": "08-07-2024 20:11:40"}, {"transaction_id": "20", "account_id": "1", "transaction_type": "transfer in", "amount": 100.15825967221213, "transaction_date": "08-07-2024 20:11:43"}, {"transaction_id": "23", "account_id": "1"

In [25]:
#!pip install phidata
#!pip install ollama
#!pip install groq
#!pip install duckduckgo-search
#!pip install markdown

In [27]:
from typing import Optional
from textwrap import dedent
from typing import Any, List

from phi.assistant import Assistant
from phi.llm.ollama import Ollama, OllamaTools
from phi.llm.groq import Groq
from phi.tools.duckduckgo import DuckDuckGo
import markdown

In [28]:
def get_local_assistant(
    llm_provider: str = "groq",
    llm_model: str = None,
    tools: list = None,
    user_id: Optional[str] = None,
    run_id: Optional[str] = None,
    debug_mode: bool = True,
) -> Assistant:
    """Get a Local Autonomous Assistant."""
    
    if llm_provider == "groq":
        llm = Groq(model=llm_model)
    elif llm_provider == "ollama":
        llm = Ollama(model=llm_model)

    assistant = Assistant(
        name="local_assistant",
        run_id=run_id,
        user_id=user_id,
        llm=llm,
        tools=tools,
        show_tool_calls=False,
        markdown=True,
        add_datetime_to_instructions=True,
        debug_mode=debug_mode,
    )
    assistant.add_introduction(
        dedent("""Hi, I'm a local AI Assistant that uses function calling to answer questions""")
    )
    return assistant

In [29]:
session_state = dict()
session_state["assistant"] = None

In [30]:
def restart_llm(provider, llm, tools, debug_mode):
    # Get the assistant
    session_state["assistant"] = get_local_assistant(
        provider,
        llm_model=llm,
        tools=tools,
        debug_mode=debug_mode
    )
    # Load existing messages
    assistant_chat_history = session_state["assistant"].memory.get_chat_history()
    if len(assistant_chat_history) > 0:
        print("Loading chat history ...")
        session_state["messages"] = assistant_chat_history
    else:
        print("No chat history found.")
        session_state["messages"] = [{"role": "assistant", "content": "Ask me questions..."}]

In [31]:
def prompt(q):
    session_state["messages"].append({"role": "user", "content": q})
    response = ""
    for delta in session_state["assistant"].run(q, stream=False):
        response += delta  # type: ignore
        markdown.markdown(response)
    # response = session_state["assistant"].print_response(q, markdown=True, stream=False)
    session_state["messages"].append({"role": "assistant", "content": response})
    print(session_state["messages"])

In [32]:
# tools = [
#     get_dates_last_n_hours,
#     get_dates_last_n_days,
#     get_dates_from_to,
#     get_account_balance,
#     withdraw_money,
#     get_bank_statement
# ]
tools = [
    get_account_balance,
    withdraw_money,
    get_bank_statement
]

In [33]:
accounts

[Account 1: 1234567890 (savings) - Balance: 1459.413211486812,
 Account 2: 1234567891 (savings) - Balance: 629.5189306765942,
 Account 3: 1234567892 (savings) - Balance: 0.0,
 Account 4: 1234567893 (savings) - Balance: 503.07099642719015]

In [34]:
#restart_llm("groq", "mixtral-8x7b-32768", tools, False)
restart_llm("ollama", "dolphin-mixtral:8x7b", tools, False)
prompt("Is there money in account with id 4")

Loading chat history ...
[{'role': 'assistant', 'content': "Hi, I'm a local AI Assistant that uses function calling to answer questions", 'metrics': {}}, {'role': 'user', 'content': 'Is there money in account with id 4'}, {'role': 'assistant', 'content': ' Yes, there is money in account with id 4. The current balance is $503.07.'}]


In [35]:
# restart_llm("ollama", "dolphin-mixtral:8x7b", tools, False)
# prompt("What is tha balance in account 2")

In [35]:
accounts

[Account 1: 1234567890 (savings) - Balance: 1459.413211486812,
 Account 2: 1234567891 (savings) - Balance: 629.5189306765942,
 Account 3: 1234567892 (savings) - Balance: 0.0,
 Account 4: 1234567893 (savings) - Balance: 503.07099642719015]

In [36]:
# restart_llm("ollama", "dolphin-mixtral:8x7b", tools, False)
prompt("withdraw 44 dollars from account with id 3")

Insufficient funds. Current balance is 0.0
[{'role': 'assistant', 'content': "Hi, I'm a local AI Assistant that uses function calling to answer questions", 'metrics': {}}, {'role': 'user', 'content': 'Is there money in account with id 4'}, {'role': 'assistant', 'content': ' Yes, there is money in account with id 4. The current balance is $503.07.'}, {'role': 'user', 'content': 'withdraw 44 dollars from account with id 3'}, {'role': 'assistant', 'content': ' <bos_response>\n{\n    "tool_calls": [\n        {\n            "name": "get_account_balance",\n            "arguments": {\n                "account_id": "3"\n            }\n        },\n        {\n            "name": "withdraw_money",\n            "arguments": {\n                "account_id": "3",\n                "amount": 44\n            }\n        }\n    ]\n}\n</bos_response>'}]


In [38]:
accounts

[Account 1: 1234567890 (savings) - Balance: 1700.661680898512,
 Account 2: 1234567891 (savings) - Balance: 280.26055165681476,
 Account 3: 1234567892 (savings) - Balance: 269.0854582501533,
 Account 4: 1234567893 (savings) - Balance: 572.401768003466]

In [37]:
#restart_llm("ollama", "dolphin-mixtral:8x7b", tools, False)
prompt("Retrieve a bank statement for account 2 for the past 2 days")

{'start_date': '06-07-2024 20:15:27', 'end_date': '08-07-2024 20:15:27'}
Bank Statement for Account 2
Period: 2024-07-06 20:15:27 - 2024-07-08 20:15:27
Starting Balance: 0.0
Ending Balance: 629.5189306765942
Transactions:
  Transaction 1: Account 2 - deposit - Amount: 398.2385802411794 - Date: 2024-07-08 20:11:23.225865
  Transaction 2: Account 2 - withdrawal - Amount: 147.00793147884642 - Date: 2024-07-08 20:11:24.226165
  Transaction 3: Account 2 - withdrawal - Amount: 186.09453018131686 - Date: 2024-07-08 20:11:25.226458
  Transaction 4: Account 2 - deposit - Amount: 481.31525634724414 - Date: 2024-07-08 20:11:28.228332
  Transaction 5: Account 2 - deposit - Amount: 395.0269433761185 - Date: 2024-07-08 20:11:30.229454
  Transaction 9: Account 2 - withdrawal - Amount: 148.7553797141893 - Date: 2024-07-08 20:11:32.230560
  Transaction 18: Account 2 - withdrawal - Amount: 100.15825967221213 - Date: 2024-07-08 20:11:43.235691
  Transaction 19: Account 2 - transfer out - Amount: 100.1582

In [122]:
#restart_llm("groq", "llama3-8b-8192", tools, False)
restart_llm("ollama", "dolphin-mixtral:8x7b", tools, False)
prompt("Find the bank statement in account with id 1 for the last 5 hours")

Loading chat history ...


DEBUG    *********** Assistant Run Start: a76649c8-7c55-46f8-8682-ea62fdb154af ***********

DEBUG    Loaded memory

DEBUG    Function get_account_balance added to LLM.

DEBUG    Function withdraw_money added to LLM.

...

DEBUG    ============== assistant ==============

DEBUG    Here is my response:                                                                                      
                                                                                                                   
         The bank statement for account ID 1 for the last 5 hours is as follows:                                   
                                                                                                                   
         {"account_id": "1", "statement": {"account_id": "1", "start_date": "02-07-2024 04:46:04", "end_date":     
         "02-07-2024 09:46:04", "start_balance": 0.0, "end_balance": 2812.9597340960213, "transactions":           
         [{"transaction_id": "3", "account_id": "1", "transaction_type": "transfer in", "amount":                  
         248.76547085454925, "transaction_date": "02-07-2024 09:43:16"}, {"transaction_id": "6", "account_id": "1",
         "transaction_type": "transfer in", "amount": 258.53032026263713, "transaction_date": "02-07-2024          
         09:43:21"}, {"transaction_id": "17", "account_id": "1", "transaction_type": "transfer in", "amount":      
         492.816680644318, "transaction_date": "02-07-2024 09:43:24"}, {"transaction_id": "20", "account_id": "1", 
         "transaction_type": "transfer in", "amount": 381.67779475901114, "transaction_date": "02-07-2024          
         09:43:25"}, {"transaction_id": "23", "account_id": "1", "transaction_type": "transfer in", "amount":      
         196.64918453292358, "transaction_date": "02-07-2024 09:43:27"}, {"transaction_id": "24", "account_id":    
         "1", "transaction_type": "deposit", "amount": 107.03827387170688, "transaction_date": "02-07-2024         
         09:43:28"}, {"transaction_id": "30", "account_id": "1", "transaction_type": "transfer in", "amount":      
         378.7741924952769, "transaction_date": "02-07-2024 09:43:32"}, {"transaction_id": "31", "account_id": "1",
         "transaction_type": "deposit", "amount": 207.57263164870733, "transaction_date": "02-07-2024 09:43:34"},  
         {"transaction_id": "35", "account_id": "1", "transaction_type": "transfer in", "amount":                  
         148.83667203734672, "transaction_date": "02-07-2024 09:43:36"}, {"transaction_id": "38", "account_id":    
         "1", "transaction_type": "transfer in", "amount": 147.31742703338318, "transaction_date": "02-07-2024     
         09:43:37"}, {"transaction_id": "43", "account_id": "1", "transaction_type": "transfer in", "amount":      
         411.82342953356, "transaction_date": "02-07-2024 09:43:40"}, {"transaction_id": "45", "account_id": "1",  
         "transaction_type": "withdrawal", "amount": 199.59334325877614, "transaction_date": "02-07-2024           
         09:43:42"}, {"transaction_id": "46", "account_id": "1", "transaction_type": "transfer out", "amount":     
         199.59334325877614, "transaction_date": "02-07-2024 09:43:42"}, {"transaction_id": "49", "account_id":    
         "1", "transaction_type": "deposit", "amount": 118.51040886271932, "transaction_date": "02-07-2024         
         09:43:44"}, {"transaction_id": "50", "account_id": "1", "transaction_type": "transfer out", "amount":     
         118.51040886271932, "transaction_date": "02-07-2024 09:43:44"}]}

DEBUG    ---------- Ollama Response End ----------

DEBUG    *********** Assistant Run End: a76649c8-7c55-46f8-8682-ea62fdb154af ***********

[{'role': 'assistant', 'content': "Hi, I'm a local AI Assistant that uses function calling to answer questions", 'metrics': {}}, {'role': 'user', 'content': 'Find the bank statement in account with id 1 for the last 5 hours'}, {'role': 'assistant', 'content': 'Here is my response:\n\nThe bank statement for account ID 1 for the last 5 hours is as follows:\n\n{"account_id": "1", "statement": {"account_id": "1", "start_date": "02-07-2024 04:46:04", "end_date": "02-07-2024 09:46:04", "start_balance": 0.0, "end_balance": 2812.9597340960213, "transactions": [{"transaction_id": "3", "account_id": "1", "transaction_type": "transfer in", "amount": 248.76547085454925, "transaction_date": "02-07-2024 09:43:16"}, {"transaction_id": "6", "account_id": "1", "transaction_type": "transfer in", "amount": 258.53032026263713, "transaction_date": "02-07-2024 09:43:21"}, {"transaction_id": "17", "account_id": "1", "transaction_type": "transfer in", "amount": 492.816680644318, "transaction_date": "02-07-202

In [76]:
# restart_llm("ollama", "llama3:instruct", tools)
# prompt("Find the bank statement in account 1 for the last 10 hours")

In [77]:
# restart_llm("ollama", "gemma:instruct", tools)
# prompt("Retrieve a bank statement for account 2 for the past 2 days")

In [479]:
import os 
from typing import Optional, List
from phi.assistant import Assistant
from phi.knowledge import AssistantKnowledge
from phi.llm.ollama import Ollama
from phi.embedder.ollama import OllamaEmbedder
from phi.vectordb.pgvector import PgVector2
from phi.storage.assistant.postgres import PgAssistantStorage
from phi.document import Document
from phi.document.reader.pdf import PDFReader
from phi.document.reader.website import WebsiteReader
from phi.embedder.openai import OpenAIEmbedder

In [480]:
db_url = "postgresql+psycopg://ai:ai@localhost:5532/ai"

In [481]:
def get_bank_assistant(
    llm_provider: str = "groq",
    llm_model: str = None,
    embeddings_model: str = None,
    user_id: Optional[str] = None,
    run_id: Optional[str] = None,
    tools: List[Any] = None,
    debug_mode: bool = True,
) -> Assistant:
    """Get a Local RAG Assistant."""

    # Define the embedder based on the embeddings model
    embedder = OllamaEmbedder(model=embeddings_model, dimensions=4096)
    embeddings_model_clean = embeddings_model.replace("-", "_")
    if embeddings_model == "nomic-embed-text":
        embedder = OllamaEmbedder(model=embeddings_model, dimensions=768)
    elif embeddings_model == "phi3":
        embedder = OllamaEmbedder(model=embeddings_model, dimensions=3072)

    # if llm_provider == "groq":
    #     embedder = OpenAIEmbedder(model="text-embedding-3-small", dimensions=1536)

    # Define the LLM
    if llm_provider == "groq":
        llm = Groq(model=llm_model)
    elif llm_provider == "ollama":
        llm = OllamaTools(model=llm_model)
        
    # Define the knowledge base
    knowledge = AssistantKnowledge(
        vector_db=PgVector2(
            db_url=db_url,
            collection=f"bank_documents_{embeddings_model_clean}",
            embedder=embedder,
        ),
        # 5 references are added to the prompt
        num_documents=5,
    )

    return Assistant(
        name="beaks_bank_assistant",
        run_id=run_id,
        user_id=user_id,
        llm=llm,
        storage=PgAssistantStorage(table_name="local_rag_assistant", db_url=db_url),
        knowledge_base=knowledge,
        description="You are a 'Bank Assistant' and your task is to help answer the Bank's customer queries.",
        introduction="Hi, I'm an your Assistant to help with your queries.\n\n",
        instructions=[
            f"You are interacting with the user: {user_id}",
            "When the user asks a specific question, find a relevant existing tools to provide a concise and relevant answer.",
            "For generic question, search your knowledge base using the `search_knowledge_base` tool and provide a concise and relevant answer.",
            "Do not use phrases like 'based on my knowledge' or 'depending on the information'.",
            "If you did not find an answer with any of the tools simply say. 'I have no knowledge of this'",
        ],    
        # use_tools adds default tools to search the knowledge base and chat history
        use_tools=True,
        tools=tools,
        show_tool_calls=True,
        # Uncomment this setting adds chat history to the messages
        # add_chat_history_to_messages=True,
        # Uncomment this setting to customize the number of previous messages added from the chat history
        # num_history_messages=3,
        # This setting adds references from the knowledge_base to the user prompt
        add_references_to_prompt=True,
        # This setting tells the LLM to format messages in markdown
        markdown=True,
        add_datetime_to_instructions=True,
        debug_mode=debug_mode,
    )

In [482]:
# Create assistant run (i.e. log to database) and save run_id in session state
session_state["rag_assistant_run_id"] = session_state["rag_assistant"].create_run()

In [483]:
# Load knowledge base
def load_knowledge(input_url=None):
    if session_state["rag_assistant"].knowledge_base:
        # -*- Add websites to knowledge base
        session_state["processed_urls"] = []
        if (input_url is not None) and (input_url not in session_state["processed_urls"]):
            session_state["processed_urls"].append(input_url)
            
            print("Processing URLs...")
            scraper = WebsiteReader(max_links=5, max_depth=5)
            web_documents: List[Document] = scraper.read(input_url)
            if web_documents:
                session_state["rag_assistant"].knowledge_base.load_documents(web_documents, upsert=True)
            else:
                print("Could not read website") 
    
        # Add PDFs to knowledge base
        session_state["processed_pdfs"] = []
        uploaded_file = None
        if uploaded_file is not None:
            print("Processing PDF...")
            rag_name = uploaded_file.name.split(".")[0]
            if rag_name not in session_state["processed_pdfs"]:
                reader = PDFReader()
                rag_documents: List[Document] = reader.read(uploaded_file)
                if rag_documents:
                    session_state["rag_assistant"].knowledge_base.load_documents(rag_documents, upsert=True)
                else:
                    print("Could not read PDF")

In [484]:
def prompt(q):
    session_state["messages"].append({"role": "user", "content": q})
    response = ""
    for delta in session_state["rag_assistant"].run(q, stream=False):
        response += delta  # type: ignore
        markdown.markdown(response)
    session_state["messages"].append({"role": "assistant", "content": response})
    print(session_state["messages"])

In [485]:
# clear knowledgebase
# session_state["rag_assistant"].knowledge_base.vector_db.clear()

In [486]:
def restart_llm(provider, llm, embeddings_model, tools):
    # Get the assistant
    session_state["assistant"] = get_bank_assistant(
        provider,
        llm_model=llm,
        embeddings_model=embeddings_model,
        tools=tools
    )
    
    if session_state["rag_assistant"].storage:
        rag_assistant_run_ids: List[str] = session_state["rag_assistant"].storage.get_all_run_ids()
        print(f"rag_assistant_run_ids: {rag_assistant_run_ids}")
        new_rag_assistant_run_id = rag_assistant_run_ids[0] 
        if session_state["rag_assistant_run_id"] != new_rag_assistant_run_id:
            print(f"---*--- Loading run: {new_rag_assistant_run_id} ---*---")
            session_state["rag_assistant"] = get_rag_assistant(
                llm_model="llama3:instruct", embeddings_model=embeddings_model, run_id=new_rag_assistant_run_id
            )
    
    # Load existing messages
    assistant_chat_history = session_state["rag_assistant"].memory.get_chat_history()
    if len(assistant_chat_history) > 0:
        print("Loading chat history")
        session_state["messages"] = assistant_chat_history
    else:
        print("No chat history found")
        session_state["messages"] = [{"role": "assistant", "content": "Upload a doc and ask me questions..."}]
        session_state["messages"] = [{"role": "assistant", "content": "Ask me questions..."}]

    # load_knowledge(input_url="https://www.standardbank.com/sbg/standard-bank-group/our-business/what-we-do")

In [487]:
# restart_llm("groq", "llama3-8b-8192", "nomic-embed-text", tools)
# prompt("How do i get a loan bank?")

In [488]:
restart_llm("ollama", "gemma:instruct", "nomic-embed-text", tools)
prompt("Retrieve a bank statement for account 2 for the past 2 days")

DEBUG    Debug logs enabled

rag_assistant_run_ids: ['053a8ef9-21f0-408b-b27e-a31d416ccec2', '5fbde629-2363-4243-8cbd-a8c659acc42f', '2f21f9a5-343c-4c74-8f8a-5c7986f211fd', 'f77d0311-d1d3-4f63-b5c2-b833a7a1f00d', '13657779-1e52-411c-bfdf-3fd006659928', '8b8101f0-ca2d-4147-bc2d-55d41f3a72aa']
Loading chat history


DEBUG    *********** Run Start: 053a8ef9-21f0-408b-b27e-a31d416ccec2 ***********

DEBUG    -*- Loading run: 053a8ef9-21f0-408b-b27e-a31d416ccec2

DEBUG    -*- Loaded run: 053a8ef9-21f0-408b-b27e-a31d416ccec2

...

DEBUG    ============== system ==============

DEBUG    You are a 'Bank Assistant' and your task is to help answer the Bank's customer queries.                   
         YOU MUST FOLLOW THESE INSTRUCTIONS CAREFULLY.                                                             
         <instructions>                                                                                            
         1. You are interacting with the user: None                                                                
         2. When the user asks a specific question, find a relevant existing tools to provide a concise and        
         relevant answer.                                                                                          
         3. For generic question, search your knowledge base using the `search_knowledge_base` tool and provide a  
         concise and relevant answer.                                                                              
         4. Do not use phrases like 'based on my knowledge' or 'depending on the information'.                     
         5. If you did not find an answer with any of the tools simply say. 'I have no knowledge of this'          
         6. Use markdown to format your answers.                                                                   
         7. The current time is 2024-05-02 06:30:18.397230                                                         
         </instructions>

DEBUG    ============== user ==============

DEBUG    Respond to the following message from a user:                                                             
         USER: Retrieve a bank statement for account 2 for the past 2 days                                         
                                                                                                                   
                                                                                                                   
         ASSISTANT:

NotFoundError: Error code: 404 - {'error': {'message': 'The model `llama3:instruct` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'code': 'model_not_found'}}